# Raise your hand if you are not here:
<img src="raisehand2.png" style="width: 400px;"/>

### What we have for this investigation:
- a couple concepts...
- a couple graphs to visualize missing data...
- three simple imputation examples...
- testing the imputed datasets...
- categorical feature...

### Assume you have invistigated into the reasons why values are missing and have decided what to do...
### Univariate vs Multivariate imputation
From lecture, we learned two missing value handling strategies 
1. dropping rows with missing values 
2. fill missing values with mean, median, etc <br>
Strategy 2 is what we call a univariate imputation where you use the observed values from the feature to fill in the missing values. <br>
**BUT... we could also use the entire set of features in the dataset to estimate the missing values in a feature!**

### How does multivariate imputation work?
For a feature with missing values, its observed values (as the output) is regressed on the other features (as the input) in the data, and then the regressor is used to predict the missing values. This is done for each feature with missing values in an iterative fashion. The results of the final round of imputation are returned. <br>
1. Missing values are filled in randomly from observed values
2. Apply regressor to impute feature one at a time
3. Terminate iteration when the average imputed values converge

<img src="mice_flow_chart.png" style="width: 750px;"/>

### Also just so you know, there are Single vs. Multiple Imputations:
- If we were interested in understanding the uncertainty associated with the missing values, the dataset should be imputed multiple times and each set is used for separate analysis. 
> Imputing one value for a missing datum cannot be correct in general, because we don’t know what value to impute with certainty (if we did, it wouldn’t be missing). -- **Donald B. Rubin**
- IterativeImputer only returns the result of a single imputation (because for preditive modeling, uncertainty is less of an interest). 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris

%matplotlib inline

### Loading the iris dataset from sklearn:

In [ ]:
iris = load_iris() 
df = pd.DataFrame(data = np.c_[iris["data"], iris["target"]],\
                     columns = iris["feature_names"] + ["species"])
df.isna().sum()

In [ ]:
df.head()

### Introducing missing values in sepal_length and petal_width:

In [ ]:
df_miss = df.copy()

np.random.seed(11)
mask = np.random.randint(0, 150, size = 15)
mask2 = np.random.randint(0, 150, size = 10)

df_miss["sepal length (cm)"][mask] = np.nan
df_miss["petal width (cm)"][mask2] = np.nan
df_miss.isna().sum()

### Visualizing missing values:

In [ ]:
import missingno as msno

In [ ]:
#There is also msno.bar(df)
msno.matrix(df_miss, figsize=(10, 6))

In [ ]:
#This shows the correlation between missing values of the two features
msno.heatmap(df_miss, figsize=(5, 5))

### Imputing missing values with IterativeImputer:

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
#default estimator/regressor is Baysian Ridge
ii_imputer = IterativeImputer(max_iter=10, random_state=11)

df_ii = df_miss.copy()
df_ii.iloc[:, :] = ii_imputer.fit_transform(df_ii)

In [ ]:
df_ii.isna().sum()

### Imputing missing values with other variants of IterativeImputer:

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf = RandomForestRegressor(n_estimators=10, random_state=1)
rf_imputer = IterativeImputer(estimator=rf, random_state=17)

df_rf = df_miss.copy()
df_rf.iloc[:, :] = rf_imputer.fit_transform(df_rf)
df_rf.isna().sum()

### Imputing missing values with KNN:

In [ ]:
from sklearn.impute import KNNImputer

In [ ]:
knn_imputer = KNNImputer(n_neighbors=5, weights="uniform")

df_knn = df_miss.copy()
df_knn.iloc[:, :] = knn_imputer.fit_transform(df_knn)
df_knn.isna().sum()

### Testing the datasets:

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
# mask = (df_miss["sepal length (cm)"].isna()) | (df_miss["petal width (cm)"].isna())
# df_miss[mask]
df_miss = df_miss.dropna()
datasets = [df, df_miss, df_ii, df_knn, df_rf]
dfnames = ["original", "drop missing", "iterative imputed", "knn imputed", "Random Forests imputed"]

In [ ]:
logit = LogisticRegression(C = 0.9)
for i in range(len(datasets)):
    X_train, X_test, y_train, y_test = train_test_split(datasets[i].iloc[:, 0:4], datasets[i]["species"], \
                                                        test_size=0.3, random_state=41)
    logit.fit(X_train, y_train)
    print(f"Scores for the {dfnames[i]} dataset are")
    print("Training: {:6.2f}%".format(100*logit.score(X_train, y_train)))
    print("Test: {:6.2f}%".format(100*logit.score(X_test, y_test)))

### Extra steps to impute categorical feature (with strings as values):

In [ ]:
#converting categorical values (strings) to numeric either with OrdinalEncoder or one_hot_encoder
from sklearn.preprocessing import OrdinalEncoder
feature_OE = OrdinalEncoder( )

#pull out the feature to be imputed
feature_impute = df["target_feature"]

#get all the observed categories of the feature
feture_impute_observed = feature_impute[feature_impute.notnull( )] 
  
#reshape feature_impute_notnull to a 2D array
reshape_vals = var_impute_notnull.values.reshape(-1, 1)

#convert levels of the categorical feature to numbers
encoded_vals = feature_OE.fit_transform(reshape_vals)

#put the transformed values back to the dataset
data.loc[feature_impute.notnull( ), "target_feature"] = np.squeeze(encoded_vals)

In [ ]:
#imputation, and round imputed values to integers since categories are represented by integers
ii_imputer = IterativeImputer(max_iter=10, random_state=11)
df.iloc[:, :] = np.round(ii_imputer.fit_transform(df))

In [ ]:
#reverse the transformation
levels = df["target_feature"].values.reshape(-1, 1)
df["target_feature"] = feature_OE.inverse_transform(levels)

### Questions? <br> More specifics of the functions used in the demo can be found from sklearn:
<img src="friends_minion.png" style="width: 300px;"/>

### References:
1. https://scikit-learn.org/stable/modules/impute.html#multivariate-feature-imputation
2. https://learn.datacamp.com/courses/dealing-with-missing-data-in-python
3. https://cran.r-project.org/web/packages/miceRanger/vignettes/miceAlgorithm.html